In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from sklearn.preprocessing import PowerTransformer, StandardScaler,  MinMaxScaler, Normalizer, QuantileTransformer, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [6]:
def dfclean(x):
    x.rename(columns=dict(zip(x.columns,[i.replace(" ","_").lower() for i in x.columns])),inplace=True)
    x.months_since_last_claim.fillna(np.median(x.months_since_last_claim.drop(list(x[x.months_since_last_claim.isna()].index),axis=0)),inplace=True)
    x.number_of_open_complaints.fillna(0, inplace=True)
    x.response.fillna('no', inplace=True)
    x.drop(list(x[x.state.isna() | x.vehicle_size.isna() | x.vehicle_class.isna()].index),axis=0,inplace=True)
    x['month']=pd.to_datetime(x.effective_to_date, format='%m/%d/%y').dt.month
    i=x['total_claim_amount']
    y=x._get_numeric_data().drop(['total_claim_amount'],axis=1)
    z=x.select_dtypes(object).drop(['customer','effective_to_date'],axis=1)
    df=preprocess(y,z)
    return x,df,i


def preprocess(n,c):
    encoder=OneHotEncoder(drop='first').fit(c)
    c=encoder.transform(c)
    c=pd.DataFrame(c.toarray(), columns=encoder.get_feature_names_out())
    transformer=PowerTransformer(standardize=True).fit(n)
    num=transformer.transform(n)
    scaler=MinMaxScaler().fit(n)
    num=pd.DataFrame(scaler.transform(n))

    return pd.concat([n,c],axis=1)





In [7]:
def metric(y,x):
    
    for i in (mean_absolute_error,r2_score,mean_squared_error):
        sol=str(i)[10:str(i).find(' ',10)].replace('_',' ')
        i(y,x)
        print(sol,'is',i(y,x))
        if i==mean_squared_error:
            print('root',sol,'is',np.sqrt(i(y,x)))

cust_da, finaldf, target=  dfclean(pd.read_csv('files_for_lab\csv_files\marketing_customer_analysis.csv'))


X_train, X_test, y_train, y_test = train_test_split(finaldf,target,test_size=0.2,random_state=42)
lm = LinearRegression().fit(X_train,y_train)

In [13]:

print('For train set:')
metric(y_train,lm.predict(X_train))

print('\nFor test set:')
metric(y_test,lm.predict(X_test))


For train set:
mean absolute error is 95.20252547123656
r2 score is 0.7743092025940634
mean squared error is 19308.21890396365
root mean squared error is 138.9540172285913

For test set:
mean absolute error is 94.51732465512121
r2 score is 0.7626561666853123
mean squared error is 18871.455206846804
root mean squared error is 137.37341521141127
